In [3]:
import pandas as pd
import numpy as np
import scipy

In [4]:
from scipy import spatial
from sklearn import preprocessing


In [5]:
reps = pd.read_excel('data.xlsx', sheet_name='Reps Profiling')
reps.head()

,Unnamed: 0,Unnamed: 0.1,First Name,Last Name,District,Party,Room,Phone Number,Email Address,Link,Name,County,% Dem,% Rep,Weeks in Office,No. Enviro
0,0,0,James,Arciero,2nd Middlesex,Democrat,277,(617) 722-2012,James.Arciero@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,James Arciero,Middlesex County,0.663227,0.281705,572,5
1,1,1,Brian,Ashe,2nd Hampden,Democrat,236,(617) 722-2430,Brian.Ashe@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Brian Ashe,Hampden County,0.550959,0.390720,572,1
2,2,2,Bruce,Ayers,1st Norfolk,Democrat,167,(617) 722-2230,Bruce.Ayers@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Bruce Ayers,Norfolk County,0.612467,0.333084,1092,14
3,3,3,Ruth,Balser,12th Middlesex,Democrat,167,(617) 722-2810,Ruth.Balser@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Ruth Balser,Middlesex County,0.663227,0.281705,1092,6
4,4,4,Christine,Barber,34th Middlesex,Democrat,254,(617) 722-2220,Christine.Barber@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Christine Barber,Middlesex County,0.663227,0.281705,288,0


In [6]:
bills = pd.read_excel('data.xlsx', sheet_name="Bill Filtering")
bills.head()

,Unnamed: 0,Name,Link,H2836,H2007
0,0,James Arciero,https://malegislature.gov/Legislators/Profile/...,1,0
1,1,Brian Ashe,https://malegislature.gov/Legislators/Profile/...,0,0
2,2,Bruce Ayers,https://malegislature.gov/Legislators/Profile/...,1,0
3,3,Ruth Balser,https://malegislature.gov/Legislators/Profile/...,1,0
4,4,Christine Barber,https://malegislature.gov/Legislators/Profile/...,1,0


# Logistic Regression using Rep features

In [54]:
import warnings
warnings.filterwarnings('ignore')

**Clean and label with H2836**

In [8]:
df = pd.read_excel('data.xlsx', sheet_name='Reps Profiling')
df.drop(['Unnamed: 0', 'First Name', 'Last Name', 'District', 'Room', 'Phone Number', 'Email Address', 'Link'], axis=1, inplace=True)
df.head()

,Unnamed: 0.1,Party,Name,County,% Dem,% Rep,Weeks in Office,No. Enviro
0,0,Democrat,James Arciero,Middlesex County,0.663227,0.281705,572,5
1,1,Democrat,Brian Ashe,Hampden County,0.550959,0.390720,572,1
2,2,Democrat,Bruce Ayers,Norfolk County,0.612467,0.333084,1092,14
3,3,Democrat,Ruth Balser,Middlesex County,0.663227,0.281705,1092,6
4,4,Democrat,Christine Barber,Middlesex County,0.663227,0.281705,288,0


**Turn categorical labels into numbers**

In [40]:
df['Party'].value_counts()

Democrat      127
Republican     32
Name: Party, dtype: int64

In [41]:
df['County'].value_counts()

Middlesex County     37
Suffolk County       19
Worcester County     18
Essex County         18
Norfolk County       15
Bristol County       14
Plymouth County      12
Hampden County       11
Barnstable County     5
Berkshire County      4
Hampshire County      3
Franklin County       2
Dukes County          1
Name: County, dtype: int64

In [55]:
def cleanDataset(df):
    #Update wrong cell
    df['Party'][155] = 'Republican'
    
    #Implement the One-Hot Encoding
    le = preprocessing.LabelEncoder()
    num_df = df[["Party", 'County']].apply(le.fit_transform)
    
    enc = preprocessing.OneHotEncoder()
    enc.fit(num_df)
    
    onehotlabels = enc.transform(num_df).toarray()
    clean_df = pd.DataFrame(onehotlabels)
    clean_df[['% Dem', '% Rep', 'Weeks in Office', 'No. Enviro']] = df[['% Dem', '% Rep', 'Weeks in Office', 'No. Enviro']]
    clean_df['Label'] = bills['H2836']

    return clean_df

clean = cleanDataset(df)
clean

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,% Dem,% Rep,Weeks in Office,No. Enviro,Label
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,572,5,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.550959,0.390720,572,1,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.612467,0.333084,1092,14,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,1092,6,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,288,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.539938,0.405843,288,2,0
155,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.641509,0.271856,288,2,0
156,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.550959,0.390720,156,0,1
157,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.584960,0.359986,468,0,0


**Training**

In [12]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression()

In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(clean, test_size=0.35, random_state=3)
y_labels_train = train["Label"]
X_class_train = train.drop("Label", axis=1)
y_labels_test = test["Label"]
X_class_test = test.drop("Label", axis=1)

In [14]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,% Dem,% Rep,Weeks in Office,No. Enviro,Label
45,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.729229,0.216372,156,13,1
35,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,988,0,1
134,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.518633,0.425797,468,16,0
50,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.612467,0.333084,1508,4,1
76,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,1300,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.539938,0.405843,156,0,0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,572,5,1
131,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.662853,0.268302,80,0,1
152,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.550959,0.390720,1404,0,1


In [15]:
log.fit(X_class_train, y_labels_train)

/Users/sgrawal/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**Predictions**

In [43]:
predicts = log.predict(X_class_test)

pred_prob = log.predict_proba(X_class_test)
ones_prob = [x[1] for x in pred_prob]

In [17]:
pred_prob[:5]

array([[0.29828168, 0.70171832],
       [0.42725514, 0.57274486],
       [0.33634912, 0.66365088],
       [0.3702728 , 0.6297272 ],
       [0.29828168, 0.70171832]])

In [18]:
ones_prob[:5]

[0.7017183159366296,
 0.5727448632229833,
 0.6636508800152003,
 0.6297271969597913,
 0.7017183159366296]

In [19]:
results = pd.DataFrame({"log":predicts, "prob_one":ones_prob, "actual":y_labels_test})
results.head()

,log,prob_one,actual
51,1,0.701718,1
3,1,0.572745,1
40,1,0.663651,1
31,1,0.629727,0
70,1,0.701718,0


## Overall Accuracy

In [20]:
from sklearn.model_selection import cross_val_score

y = clean["Label"]
X = clean.drop("Label", axis=1)

In [21]:
new_log = LogisticRegression()

In [56]:
cvs = cross_val_score(new_log, X, y, cv=5)
cvs

array([0.75      , 0.625     , 0.84375   , 0.6875    , 0.80645161])

In [44]:
np.mean(cvs)

0.7425403225806452

In [45]:
len_test = len(results.index)
len_train = len(X_class_train.index)
num_correct = len(results[results['log']==results['actual']].index)
num_correct/len_test

0.7861635220125787

In [47]:
clean.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,% Dem,% Rep,Weeks in Office,No. Enviro,Label
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,572,5,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.550959,0.390720,572,1,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.612467,0.333084,1092,14,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,1092,6,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,288,0,1


# Putting into Excel with all Predictions

In [48]:
log = LogisticRegression()
log.fit(X_class_train, y_labels_train)

/Users/sgrawal/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
predicts = log.predict(X) # run on entire dataset to get all scores
pred_prob = log.predict_proba(X)
ones_prob = [x[1] for x in pred_prob]

In [50]:
ones_prob[:5]

[0.7045424808227657,
 0.7034621920102618,
 0.600063926469715,
 0.5727448632229833,
 0.7762331738917433]

In [51]:
results = pd.DataFrame({"log":predicts, "prob_one":ones_prob, "actual":y})
results

,log,prob_one,actual
0,1,0.704542,1
1,1,0.703462,0
2,1,0.600064,1
3,1,0.572745,1
4,1,0.776233,1
...,...,...,...
154,0,0.060174,0
155,0,0.087432,0
156,1,0.790396,1
157,0,0.036633,0


In [57]:
cross_val_score(LogisticRegression(), X, y, cv=5)

array([0.75      , 0.625     , 0.84375   , 0.6875    , 0.80645161])

In [58]:
np.mean(cross_val_score(LogisticRegression(), X, y, cv=5))

0.7425403225806452

In [59]:
len_all = len(results.index)
num_correct = len(results[results['log']==results['actual']].index)
num_correct/len_all

0.7861635220125787

In [33]:
num_correct

125

In [60]:
len_all

159

In [61]:
results['prob_one']

0      0.704542
1      0.703462
2      0.600064
3      0.572745
4      0.776233
         ...   
154    0.060174
155    0.087432
156    0.790396
157    0.036633
158    0.678846
Name: prob_one, Length: 159, dtype: float64

In [70]:
reps = pd.read_excel('data.xlsx', sheet_name='Reps Profiling')
reps['Score'] = pd.Series(results['prob_one'])
reps['Label'] = bills['H2836']
reps

,Unnamed: 0,Unnamed: 0.1,First Name,Last Name,District,Party,Room,Phone Number,Email Address,Link,Name,County,% Dem,% Rep,Weeks in Office,No. Enviro,Score,Label
0,0,0,James,Arciero,2nd Middlesex,Democrat,277,(617) 722-2012,James.Arciero@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,James Arciero,Middlesex County,0.663227,0.281705,572,5,0.704542,1
1,1,1,Brian,Ashe,2nd Hampden,Democrat,236,(617) 722-2430,Brian.Ashe@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Brian Ashe,Hampden County,0.550959,0.390720,572,1,0.703462,0
2,2,2,Bruce,Ayers,1st Norfolk,Democrat,167,(617) 722-2230,Bruce.Ayers@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Bruce Ayers,Norfolk County,0.612467,0.333084,1092,14,0.600064,1
3,3,3,Ruth,Balser,12th Middlesex,Democrat,167,(617) 722-2810,Ruth.Balser@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Ruth Balser,Middlesex County,0.663227,0.281705,1092,6,0.572745,1
4,4,4,Christine,Barber,34th Middlesex,Democrat,254,(617) 722-2220,Christine.Barber@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Christine Barber,Middlesex County,0.663227,0.281705,288,0,0.776233,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,154,154,Timothy,Whelan,1st Barnstable,Republican,542,(617) 722-2488,Timothy.Whelan@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Timothy Whelan,Barnstable County,0.539938,0.405843,288,2,0.060174,0
155,155,155,Susannah,Whipps,2nd Franklin,Unenrolled,540,(617) 722-2090,Susannah.Whipps@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Susannah Whipps,Franklin County,0.641509,0.271856,288,2,0.087432,0
156,156,156,Bud,Williams,11th Hampden,Democrat,160,(617) 722-2304,Bud.Williams@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Bud Williams,Hampden County,0.550959,0.390720,156,0,0.790396,1
157,157,157,Donald,Wong,9th Essex,Republican,541,(617) 722-2488,Donald.Wong@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Donald Wong,Essex County,0.584960,0.359986,468,0,0.036633,0


In [63]:
with pd.ExcelWriter('Final.xlsx') as writer:
    reps.to_excel(writer)

## Into Json

In [69]:
import json

In [97]:
by_name = pd.read_excel("Final.xlsx")
by_score = pd.read_excel("Final.xlsx")

In [98]:
by_name.index = df['Name']
by_score.index = df['Score']


by_name.drop(['ID', 'Room', 'County','Weeks in Office'], axis=1, inplace=True)
by_score.drop(['ID', 'Room', 'County','Weeks in Office'], axis=1, inplace=True)

In [99]:
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [101]:
name = by_name.to_dict(orient='index')
jprint(name)

{
    "Aaron Michlewitz": {
        "% Dem": 0.794723195076,
        "% Rep": 0.164873796196,
        "District": "3rd Suffolk",
        "Email Address": "Aaron.M.Michlewitz@mahouse.gov",
        "First Name": "Aaron",
        "Label": 0,
        "Last Name": "Michlewitz",
        "Link": "https://malegislature.gov/Legislators/Profile/AMM1",
        "Name": "Aaron Michlewitz",
        "No. Enviro": 2,
        "Party": "Democrat",
        "Phone Number": "(617) 722-2990",
        "Score": 0.6709907935804016
    },
    "Aaron Vega": {
        "% Dem": 0.55095894516,
        "% Rep": 0.390720207771,
        "District": "5th Hampden",
        "Email Address": "Aaron.Vega@mahouse.gov",
        "First Name": "Aaron",
        "Label": 1,
        "Last Name": "Vega",
        "Link": "https://malegislature.gov/Legislators/Profile/A_V1",
        "Name": "Aaron Vega",
        "No. Enviro": 1,
        "Party": "Democrat",
        "Phone Number": "(617) 722-2011",
        "Score": 0.742409213678371

In [108]:
with open("byName.json", "w") as write_file:
    # dump takes in data, then place to put it
    json.dump(name, write_file)

In [111]:
import json # py library
file = open('byName.json', 'r')
dic = json.load(file)

{'James Arciero': {'First Name': 'James',
  'Last Name': 'Arciero',
  'District': '2nd Middlesex',
  'Party': 'Democrat',
  'Phone Number': '(617) 722-2012',
  'Email Address': 'James.Arciero@mahouse.gov',
  'Link': 'https://malegislature.gov/Legislators/Profile/J_A1',
  'Name': 'James Arciero',
  '% Dem': 0.663227499782,
  '% Rep': 0.2817054306,
  'No. Enviro': 5,
  'Score': 0.7045424808227657,
  'Label': 1},
 'Brian Ashe': {'First Name': 'Brian',
  'Last Name': 'Ashe',
  'District': '2nd Hampden',
  'Party': 'Democrat',
  'Phone Number': '(617) 722-2430',
  'Email Address': 'Brian.Ashe@mahouse.gov',
  'Link': 'https://malegislature.gov/Legislators/Profile/BMA1',
  'Name': 'Brian Ashe',
  '% Dem': 0.55095894516,
  '% Rep': 0.390720207771,
  'No. Enviro': 1,
  'Score': 0.7034621920102618,
  'Label': 0},
 'Bruce Ayers': {'First Name': 'Bruce',
  'Last Name': 'Ayers',
  'District': '1st Norfolk',
  'Party': 'Democrat',
  'Phone Number': '(617) 722-2230',
  'Email Address': 'Bruce.Ayers@m